# Setup.

In [1]:
# Imports.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# `create_map`: Create map with focusing on a specified city and zoom ratio.

In [2]:
from mpl_toolkits.basemap import Basemap

def create_map(city, zoom_ratio=0.8):
    """
    Description: Creates a Basemap instance for visualizing different regions of South Korea.
    Args:
        city        = str, the region to focus on. ['entire', 'seoul', 'busan', 'daejeon'].
        zoom_ratio  = float, factor to zoom in or out. Default is 1.0 (original focus).
                     Values > 1.0 zoom out, values < 1.0 zoom in.
    Returns: 
        m = Basemap instance, configured for the specified region.
    """
    # Define the base coordinates for each city
    coordinates = {
        'Korea': (34.0, 38.5, 125.5, 130.0),
        'Seoul': (36.5, 38.5, 126.5, 128.0),
        'Busan': (34.0, 36.0, 127.5, 129.5),
        'Daejeon': (35.5, 37.0, 126.5, 128.5)
    }
    
    if city not in coordinates:
        raise ValueError("Invalid city. Valid options are 'Korea', 'Seoul', 'Busan', or 'Daejeon'.")

    # Get base coordinates
    llcrnrlat, urcrnrlat, llcrnrlon, urcrnrlon = coordinates[city]
    
    # Adjust boundaries based on zoom ratio
    lat_center = (llcrnrlat + urcrnrlat) / 2
    lon_center = (llcrnrlon + urcrnrlon) / 2
    lat_span = (urcrnrlat - llcrnrlat) / zoom_ratio
    lon_span = (urcrnrlon - llcrnrlon) / zoom_ratio

    # Recalculate corners
    llcrnrlat = lat_center - lat_span / 2
    urcrnrlat = lat_center + lat_span / 2
    llcrnrlon = lon_center - lon_span / 2
    urcrnrlon = lon_center + lon_span / 2

    # Create the Basemap instance
    m = Basemap(
        projection='merc',
        llcrnrlat=llcrnrlat,
        urcrnrlat=urcrnrlat,
        llcrnrlon=llcrnrlon,
        urcrnrlon=urcrnrlon,
        resolution='i'
    )

    return m

# `plot_orders`: plot orders into the Korean map, focusing on specified `city`.

In [5]:
def plot_orders(df, city='Korea', zoom_ratio=1.0, is_main_cities=True, color_by_cluster=False, 
                figsize=(15,8), title="No Title", cmap='viridis'):
    """
    Description: Simple method to plot orders on a Korean map with optional clustering visualization.
    Args:
        df                = pd.DataFrame, where df['longitude'] and df['latitude'] are set.
        city              = city to focus. ['Korea', 'Seoul', 'Busan', 'Daejeon'].
        zoom_ratio        = float, zoom level for the map. Default is 1.0.
        is_main_cities    = if 'True', draws some main cities. ONLY works when `city`='Korea'.
        color_by_cluster  = if 'True', colors points by cluster (k-means or hierarchical).
        figsize           = (x, y). figsize for pyplot.
        title             = str to specify title of the plot.
        cmap              = str, colormap to use.
    Returns: None.
    """
    # Imports.
    import matplotlib.pyplot as plt
    
    # Map size
    plt.figure(figsize=figsize)
    
    # Create a Basemap instance focused on the specified city
    m = create_map(city, zoom_ratio=zoom_ratio)
    
    # Draw map boundaries, coastlines, and country borders
    m.drawcoastlines()
    m.drawcountries()
    m.drawmapboundary(fill_color='lightblue')
    m.fillcontinents(color='lightgreen', lake_color='lightblue')
    
    # Plot main cities (if applicable)
    if is_main_cities and city == 'Korea':
        city_data = [
            ("Seoul", 37.5665, 126.9780),   # Seoul
            ("Busan", 35.1796, 129.0756),   # Busan
            ("Daejeon", 36.3504, 127.3845)  # Daejeon
        ]
        for city_name, lat, lon in city_data:
            xx, yy = m(lon, lat)  # Convert latitude and longitude to map coordinates
            plt.text(xx - 50000, yy, city_name, fontsize=12, ha='right', color='black')  # Adjust text position
            m.scatter(xx, yy, marker='x', color='blue', zorder=5, s=100)  # Blue X marker
    
    # Convert latitude and longitude to map coordinates
    x, y = m(df['longitude'].values, df['latitude'].values)
    
    # Plot orders or clusters
    if color_by_cluster:
        cluster_labels = df['cluster']
        
        # Normalize cluster labels to ensure colorbar scaling is correct
        scatter = m.scatter(
            x, y,
            c=cluster_labels,
            cmap=cmap,
            s=10,
            alpha=0.7,
            zorder=5,
            vmin=cluster_labels.min(),  # Set minimum value for the colorbar
            vmax=cluster_labels.max()   # Set maximum value for the colorbar
        )
        plt.colorbar(scatter, label='Cluster')
    else:
        # Default red color for orders
        m.scatter(x, y, marker='o', color='red', zorder=5, s=10)
    
    # Title and display the plot
    plt.title(title + f' (City: {city})')
    plt.show()


# `plot_orders_by_month`: plot orders by month into the Korean map, creating subplots for each month.

In [9]:
def plot_orders_by_month(df, city='Korea', zoom_ratio=1.0, is_main_cities=True):
    """
    Description: Create subplots to plot orders for each month in Korean map.
    Args:
        df              = pd.DataFrame, where df['longitude'], df['latitude'], and df['date'] are set.
        city            = city to focus. ['Korea', 'Seoul', 'Busan', 'Daejeon']
        is_main_cities  = if 'True', draws some main cities. ONLY works when `city`='Korea'.
    Returns: None.
    """
    # Imports.
    import matplotlib.pyplot as plt
    
    # Convert the 'date' column to datetime format and extract month.
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month

    # Create a sorted list of unique months in the data.
    months = sorted(df['month'].unique())
    n_months = len(months)

    # Create subplots with 4 columns per row.
    n_cols = 4
    n_rows = -(-n_months // n_cols)  # Ceiling division
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))
    axes = axes.flatten()  # Flatten axes for easy indexing

    for i, month in enumerate(months):
        # Filter data for the current month.
        month_df = df[df['month'] == month]

        # Create a Basemap instance for the specified city.
        m = create_map(city, zoom_ratio=zoom_ratio)

        # Draw map boundaries, coastlines, and country borders.
        m.drawcoastlines(ax=axes[i])
        m.drawcountries(ax=axes[i])
        m.drawmapboundary(fill_color='lightblue', ax=axes[i])
        m.fillcontinents(color='lightgreen', lake_color='lightblue', ax=axes[i])

        # Plot some main cities if enabled and city is 'Korea'.
        if is_main_cities and city == 'Korea':
            city_data = [
                ("Seoul", 37.5665, 126.9780),   # Seoul
                ("Busan", 35.1796, 129.0756),   # Busan
                ("Daejeon", 36.3504, 127.3845)  # Daejeon
            ]
            for c, lat, lon in city_data:
                xx, yy = m(lon, lat)
                axes[i].text(xx - 50000, yy, c, fontsize=10, ha='right', color='black')
                m.scatter(xx, yy, marker='x', color='blue', zorder=5, s=50, ax=axes[i])

        # Plot orders for the current month.
        x, y = m(month_df['longitude'].values, month_df['latitude'].values)
        m.scatter(x, y, marker='o', color='red', zorder=5, s=10, ax=axes[i])

        # Set the title for the subplot.
        axes[i].set_title(f'Orders in Month: {month} (City: {city})')

    # Turn off unused axes.
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()